<html>
<div>
  <img src="https://www.engineersgarage.com/wp-content/uploads/2021/11/TCH36-01-scaled.jpg" width=360px width=auto style="vertical-align: middle;">
  <span style="font-family: Georgia; font-size:30px; color: white;"> <br/> University of Tehran <br/> AI_CA1 <br/> Spring 02 </span>
</div>
<span style="font-family: Georgia; font-size:15pt; color: white; vertical-align: middle;"> low_mist - std id: 810100186 </span>
</html>

In this notebook we are to solve a searching problem with different uninformed and informed search approaches and also compare their execution times. Approaches are:
- Uninformed search algorithms, such as:
    - BFS
    - IDS
- Informed search algorithms, such as:
    - A*
    - weighted A*

### Problem Description
We have a Graph in which every node does either contain a student or pizza or it doesn't contain anything and it's just a normal node.  
And each edge is either normal or is rickety which means if we cross it we have to wait at least a certain amount of time so it's safe again to cross. Each student has ordered a special pizza from special place and we have to go to each pizza station and get that pizza and deliver it, our goal is to find the shortest path with different search methods as mentioned.  
There are some other constraints such as the priority in which some students are not allowed to get their pizza before some other and we also can carry one pizza at a time.


# Modeling
## Graph and States
first we have to define the graph components.  
`Node`: contains the types plus it's edges.  
`Edge`: contains destination the types plus it's unsafe time list if it's rickety (it's a list that shows if we have to wait for some seconds so it's safe again to cross).

In [480]:
from __future__ import annotations
import itertools
from dataclasses import dataclass, field, fields, _MISSING_TYPE
from typing import Any, Callable, Optional
from enum import Flag, auto
from collections import deque

class EdgeType(Flag):
    NORMAL = auto()
    RICKETY = auto()

@dataclass
class Edge:
    '''used for edge in graph, note that other than
    usual information it has type and wait time (explained above) and weight'''
    destination: int
    id: int
    type: EdgeType = field(default_factory = EdgeType.NORMAL)
    wait_for: int = field(default_factory = 0)
    weight: int = field(default_factory = 0)
    
class NodeType(Flag):
    NORMAL = auto()
    PIZZA = auto()
    STUDENT = auto()

@dataclass
class Node:
    '''used as node in graph, like edge it contains usual things plus 
    extra attribute which is type'''
    edges: list[Edge]
    type: NodeType

Now that we have edges and nodes we define the graph that contains some extra information compared to normal graphs, namely, order of students, map from pizza to students and so forth.

In [481]:
FIRST_PRIORITY = 0
SECOND_PRIORITY = 1

class Graph:
    '''this class is only the graph connection of problem not agent's state'''
    def __init__(self, n : int):
        self.num_of_nodes = n
        self.nodes = [Node([], NodeType.NORMAL) for _ in range(n)]
        self.pizzas_corresponding_students: dict[int, int] = dict()
        self.priority_queues: list[tuple[int, int]] = list()
        self.students: list[int] = list()
        
    def add_edges(self, origin: int, destination: int, type: EdgeType, id: int, wait_for: int = 0, weight: int = 0): 
        self.nodes[origin].edges.append(Edge(destination, id, type, wait_for, weight)) 
        self.nodes[destination].edges.append(Edge(origin, id, type, wait_for, weight))
        
    def add_pizza_and_student(self, student_position: int, pizza_position: int):
        self.pizzas_corresponding_students[pizza_position] = student_position
        self.students.append(student_position)
        self.nodes[pizza_position].type = NodeType.PIZZA
        self.nodes[student_position].type = NodeType.STUDENT
        
    def add_priority(self, priority: tuple[int, int]):
        self.priority_queues.append((self.students[priority[FIRST_PRIORITY]], self.students[priority[SECOND_PRIORITY]]))

Now it's time to describe the states in which our agents will be during execution of search.  
This ```AgentState``` class contains information about the world such as our position and left pizzas' locations and so forth, also about agent itself like the path it has taken to get here and such things.  
The properties in our states are two types:  
* criteria in determining uniqueness  
    * `agent_position`: which shows the agent's position.  
    <!-- * `left_pizzas`: which depicts the left pizzas position and their corresponding student positions. -->  
    * `left_students`: which depicts the left students position and their corresponding pizza positions.  
    <!-- * `time_elapsed`: the amount of time (in seconds) that has passed since we started the search.   -->
    <!-- * `pizzas_carrying`: a dictionary containing the pizzas that we are carrying right now with their destination (note: it's said in problem that we can only carry one pizza at a time but we imagine that we can carry all of them and if we decide to deliver some pizza we will drop all other pizzas). -->  
    * `remain_time_edges`: a map for keep tracking of time that we have to wait.
    * `pizza_carrying`: the tuple which contains the pizza position and the student to whom we have to deliver the pizza. (if it exists)  
* not important in determining uniqueness  
    * `path`: which demonstrates the path we have taken to get here.  
    <!-- * `satisfied_students`: which indicates the students are given their pizzas.   -->
    * `priorities_left`: list of all priorities that are left and should be taken into account.  
    <!-- * `pizzas_passed`: number of pizzas that we have passed -->

In [482]:
@dataclass 
class AgentState:
    agent_position: int
    left_students: dict[int, int]
    left_priorities: list[tuple[int, int]]
    pizza_carrying : tuple[int, int] = field(default_factory = lambda: (None, None))
    path: list[int] = field(default_factory = lambda: list())
    remain_time_edges: dict[int, int] = field(default_factory = lambda: dict())
    
    def __tuple(self):
        return (self.agent_position, tuple(self.left_students), tuple(self.remain_time_edges), self.pizza_carrying)
    
    def __eq__(self, other: Any) -> bool:
        return (isinstance(other, AgentState)) and (
            self.agent_position == other.agent_position and
            self.left_students == other.left_students and
            self.remain_time_edges == other.remain_time_edges and
            self.pizza_carrying == other.pizza_carrying
        )
    
    def __hash__(self) -> int:
        return hash(self.__tuple)
    
    def __str__(self) -> str:
        return f"{self.agent_position}, {self.pizza_carrying}, {self.remain_time_edges}, {self.left_students}"

Next we define some useful functions for our agent.  
- `has_reached_goad`: simply checks whether there is any pizza left or not.
- `initial_state`: to create state which indicate where we are at start of the game.
- `actions`: this function gets an state and returns the list of all actions that is available.

**Note**: there are some optimization in the code each of them is explained or mentioned by comments

In [483]:
from copy import deepcopy, copy
STUDENT_INDEX = 1
PIZZA_INDEX = 0


class Agent:
    @staticmethod
    def has_reached_goal(state: AgentState) -> bool:
        return len(state.left_students) == 0

    @staticmethod
    def initial_state(graph: Graph, start_position: int) -> AgentState:
        return AgentState( 
            agent_position = start_position,
            left_priorities = graph.priority_queues,
            left_students = {value: key for key, value in graph.pizzas_corresponding_students.items()},
            path = [start_position])  
        
        
    @staticmethod
    def has_seen_student(graph: Graph,position: int) -> bool:
        return graph.nodes[position].type == NodeType.STUDENT

    @staticmethod
    def has_seen_pizza(graph: Graph,position: int) -> bool:
        return graph.nodes[position].type == NodeType.PIZZA
        
    @staticmethod
    def has_stayed(state: AgentState) -> bool:
        return len(state.path) > 1 and state.path[-1] == state.path[-2]
        
    @staticmethod
    def can_deliver_pizza(state: AgentState, position: int) -> bool:
        pizza, student = state.pizza_carrying
        return student in state.left_students and student == position
    
    @staticmethod
    def deliver_pizza(state: AgentState):
        pizza, student = state.pizza_carrying
        del state.left_students[student]
        state.left_priorities = list(filter(lambda x: x[FIRST_PRIORITY] != student, state.left_priorities))
        state.pizza_carrying = (None, None)
        
    @staticmethod
    def can_pick_pizza(state: AgentState, position: int, graph: Graph) -> bool:
        student = graph.pizzas_corresponding_students[position]
        return student in state.left_students and \
            not any(x[1] == student for x in state.left_priorities) and \
            state.pizza_carrying == (None, None)
              
    @staticmethod
    def pick_pizza(ans: list[AgentState], state: AgentState, graph: Graph, position: int):
        another_state = deepcopy(state)
        another_state.pizza_carrying = (position, graph.pizzas_corresponding_students[position])
        ans.append(another_state)
    
    @staticmethod
    def actions(graph: Graph, state: AgentState) -> list[AgentState]:
        ans : list[AgentState] = list()
        
        for edge in graph.nodes[state.agent_position].edges:
            
            if Agent.has_stayed(state) and edge.type != EdgeType.RICKETY:
                continue    # cause if we have stayed then we want to cross one of the rickety edges
            
            new_state = deepcopy(state)
            new_agent_position = edge.destination
            new_state.remain_time_edges = {k: (v - 1) for k, v in new_state.remain_time_edges.items() if v >= 1}
            
            if edge.type == EdgeType.RICKETY:
                if not edge.id in new_state.remain_time_edges:
                    new_state.remain_time_edges[edge.id] = edge.wait_for # we cross the edge so we update the time 
                else:
                    new_agent_position = state.agent_position # cause we won't move
                
            new_state.agent_position = new_agent_position
            new_state.path.append(new_agent_position)
            
            if not Agent.has_stayed(new_state):      
                      
                if Agent.has_seen_student(graph, new_agent_position) and \
                Agent.can_deliver_pizza(new_state, new_agent_position):
                    Agent.deliver_pizza(new_state)
                
                elif Agent.has_seen_pizza(graph, new_agent_position) and \
                    Agent.can_pick_pizza(new_state, new_agent_position, graph):
                    Agent.pick_pizza(ans, new_state, graph, new_agent_position)
                    
            ans.append(new_state)
            
        return ans

# Running algorithms
We have all the fundamentals that are needed so it's time to read files and implement the searches.  
**Note:** test files are in `assets\Tests` directory.  

In [484]:
from time import time
TIME_CUT_OFF = 20
TYPE_INDEX = 2
# every -1 is cause we start counting from 0

def read_input_file(filename: str) -> tuple[Graph, int]:
    with open(filename, 'r', encoding = 'utf-8') as file:
        input = [line.rstrip('\n') for line in file]
    return read_input(input)
    
    
def read_input(input: list[str]) -> Graph:
    iterative_input = iter(input)
    num_of_nodes, num_of_edges = map(int, next(iterative_input).split(" "))
    graph = Graph(num_of_nodes)
    
    edges_line = []
    for i in range(num_of_edges):
        origin, destination = map(lambda x: int(x) - 1, next(iterative_input).split(" "))
        edges_line.append([origin, destination, EdgeType.NORMAL, i])
        
    num_of_rickey_edges = int(next(iterative_input))
    for i in range(num_of_rickey_edges):
        num, time = map(int, next(iterative_input).split(" "))
        edges_line[num - 1][TYPE_INDEX] = EdgeType.RICKETY
        edges_line[num - 1].append(time)
        
    for edge in edges_line:
        graph.add_edges(*edge)
        
    start_position = int(next(iterative_input)) - 1    
        
    num_of_students = int(next(iterative_input))
    
    for i in range(num_of_students):
        student_node, pizza_node = map(lambda x: int(x) - 1, next(iterative_input).split(" "))
        graph.add_pizza_and_student(student_node, pizza_node)
        
    num_of_priority = int(next(iterative_input))
    for i in range(num_of_priority):
        first, second = map(lambda x: int(x) - 1, next(iterative_input).split(" "))
        graph.add_priority((first, second))
        
    return graph, start_position
    

# BFS
We will search for the goal state with a *Breadth-First Search (BFS)* in state graph.  
It's an uninformed search algorithm and explore states based on depth(time in this case)
- advantages:
    - complete
    - optimal
    - better exec-time than IDS
- disadvantages:
    - time and space complexity
***
- complexities:
    - time complexity: $O(b^d)$
    - space complexity: $O(b^d)$  
    
**Note**: `d`: depth of optimal solution.`b`: maximum branching factor.  
***
**Attention** :the goal check is done after the state is out of the queue.

In [485]:
def BFS(graph: Graph, start_position: int):    
    start = Agent.initial_state(graph, start_position)
    
    if Agent.has_reached_goal(start):
        return start
    
    frontier= []
    visited = set()
    frontier.append(start)
    start = time()
    finish = time() + TIME_CUT_OFF
    while frontier and finish > start:
        current_state = frontier.pop(0)
        if Agent.has_reached_goal(current_state):
            return current_state, len(visited)

        for next_state in Agent.actions(graph, current_state):
            if next_state in visited or next_state in frontier:
                continue
            frontier.append(next_state)

        visited.add(current_state)
        start = time()

    return None, None

# IDS
It's an uninformed search algorithm and explore states using DFS approach with limited depth (increasing in each iteration)
- advantages:
    - complete
    - optimal
    - better space usage than DFS
- disadvantages:
    - time complexity
***
- complexities
    - time complexity: $O(b^d)$
    - space complexity: $O(b*d)$  
    
**Note**: `d`: depth of optimal solution.`b`: maximum branching factor.  

***
The following is an implementation of *iterative-deepening search (IDS)*.  
IDS repeatedly uses *depth-limited search (DLS)* to run *depth-first search (DFS)* limited to a certain depth.  
The depth increases by one every loop until the goal state is found.

DLS is implemented recursively.

In [486]:

def DLS(graph: Graph, start_position: int, depth_limit: int) -> tuple[AgentState, int]:
    
    def DLS_wrapped(graph: Graph, state: AgentState, visited_states: set[AgentState], depth_limit: int, number_of_visited_states: int = 0) -> tuple[AgentState, int]:
        if depth_limit == 0:
            return None, number_of_visited_states
        
        if Agent.has_reached_goal(state):
            return state, number_of_visited_states
        
        for action in Agent.actions(graph, state):
            if not action in visited_states:
                visited_states.add(action)
                number_of_visited_states += 1
                ans, number_of_visited_states = DLS_wrapped(graph, action, visited_states, depth_limit - 1, number_of_visited_states)
                visited_states.remove(action)

                if ans is not None:
                    return ans, number_of_visited_states
                
                
        return None, number_of_visited_states
    
    
    state = Agent.initial_state(graph, start_position)
    visited_states : set[AgentState] = set()
    visited_states.add(state)
    return DLS_wrapped(graph, state, visited_states, depth_limit)

def IDS(graph: Graph, start: int, depth_limit: int = 1e6):
    start_time = time()
    finish = time() + TIME_CUT_OFF
    depth = 1
    total_visited = 0
    goal_state : AgentState = None
    while goal_state is None and depth <= depth_limit and finish > start_time:
        goal_state, currently_visited = DLS(graph, start, depth)
        total_visited += currently_visited
        depth += 1
        start_time = time()
    return goal_state, total_visited

# Weighted A*

It's an informed search algorithm and explore states using dijkstra approach.
This algorithm tries to continue the search in a more targeted way by predicting the amount of cost from the current state to the goal state.

This algorithm uses heuristics to predict the cost of continuing the route.

- features:
    - optimality: depends on heuristic
        - admissible heuristic for tree search
        - consistent heuristic for graph search

    - complete

- time complexity: number of node $f(n) <= C*$
- space complexity: $exponential$


    f(c): cost(n) + heuristic(n)

To implement A* search we need a heuristic to estimate to cost to reach to a goal state.  
The function I chose to use is defined as below:
$$ heuristic(state) = 2 \times (left\space students) + (1\space if\space we\space are\space carrying\space a\space pizza)$$
The logic behind it is so simple, we need to get to each pizza and deliver it and if we already have a pizza we have to deliver it to. So this is the minimum amount of cost left so our function is both ```admissible``` and ```consistent```.

it is ```admissible```; because
    for every state n, $h(n) <= h*(n)$
and it won't over-estimate the cost to reach the goal.


real cost is greater or equal to the cost implied by heuristic;

so, it's ```consistent``` and
    $real\space cost(n_1 -> n_2) >= h(n_1) - h(n_2)$

**attention**: $|h(n_1) - h(n_2)| <= 1$ and in each step, heuristic estimation can't change more than 1 unit.

The A* search is basically BFS but instead of queue we have priority queue
Weighted A* is just multiplying the heuristic value by ```alpha```. So for alpha=1, it is the same as A*.  
Using an alpha value too high may not keep optimality. Because a consistent heuristic is always lower than the real cost, by multiplying it by an alpha, we hope to reach values closer to the real cost but this may break its consistency and therefore not return the optimal answer.  
A* performs better than the previous algorithms in case of memory and time consumption. This is because it expands less states and is focused towards the goal state.

In [487]:
from bisect import bisect_left

def AStar(graph: Graph, start_position: int, alpha: int = 1) -> tuple[AgentState, int]:
    def insert(seq: list[AgentState], keys: list[int], item: AgentState, key: Callable[[AgentState], int] = lambda x: x):
        '''Insert an item into a sorted list using a separate corresponding
        sorted keys list and a key() to extract the key from each item.
        Based on insert() method in SortedCollection recipe:
        '''
        k = key(item)  
        i = bisect_left(keys, k)  
        keys.insert(i, k)  
        seq.insert(i, item) 
    
    def h(state: AgentState) -> int:
        '''this function only estimates the cost left'''
        return 2 * len(state.left_students) + (1 if state.pizza_carrying[0] is not None else 0)
    
    def heuristic(state: AgentState) -> int:
        '''this function is sum of what has been paid and what is likely to be paid'''
        return alpha * h(state) + len(state.path)
    
    initial_state = Agent.initial_state(graph, start_position)
    
    frontier : list[int] = list()   # note that this list is priority queue and we insert in that based on that
    visited : set[AgentState] = set()
    frontier.append(initial_state)
    keys: list[int] = list(map(heuristic, frontier))
    start_time = time()
    finish = time() + TIME_CUT_OFF
    while frontier and start_time < finish:
        new_state = frontier.pop(0)
        if Agent.has_reached_goal(new_state):
            return new_state, len(visited)
        
        for action in Agent.actions(graph, new_state):
            if action in visited or action in frontier:
                continue
            insert(frontier, keys, action, heuristic)
        
        visited.add(new_state)
        start_time = time()
    
    return None, None    

# Testing 
Ok first we put test in a directory in order to run different algorithms.
We will repeat them *TEST_REPEATS* times and calculate the average.  
Each search algorithm will also return the goal state with number of states it has visited.  

In [488]:
TEST_REPEATS = 3

def run(search_alg: Callable[[Graph, AgentState], tuple[AgentState, int]], filename: str):
    graph, start_position = read_input_file(filename)
    time_start = time()
    for _ in range(TEST_REPEATS - 2):
        goal_state, num_visited = search_alg(graph, start_position)
    time_elapsed = (time() - time_start) / TEST_REPEATS

    if goal_state is None:
        print('No solution')
    else:
        print(*[x + 1 for x in goal_state.path], sep = '->')
        print('Path cost:', len(goal_state.path) - 1)
        print('Visited states:', num_visited)
        print('Average time:', time_elapsed)

In [489]:
from functools import partial

search_algorithms ={"BFS": BFS, "IDS": IDS, "A*": partial(AStar, alpha = 1), "A* alpha = 2": partial(AStar, alpha = 2), "A* alpha = 4": partial(AStar, alpha = 4) }

NUMBER_OF_TESTS = 4

for i in range(NUMBER_OF_TESTS):
    print("---------------------------------------------------------------------------")
    print(f" **  Test Number {i}  ** ")
    for search_algorithm in search_algorithms:
        print(f"---------------{search_algorithm}-------------------")
        algorithm_function = search_algorithms[search_algorithm]
        run(algorithm_function, f"assets\Tests\Test{i}.txt")
    print("---------------------------------------------------------------------------")

---------------------------------------------------------------------------
 **  Test Number 0  ** 
---------------BFS-------------------
1->2->3->4->4->4->3->2->3->5
Path cost: 9
Visited states: 89
Average time: 0.0018420219421386719
---------------IDS-------------------
1->2->3->4->4->4->3->2->3->5
Path cost: 9
Visited states: 2123
Average time: 0.03389056523640951
---------------A*-------------------
1->2->3->4->4->4->3->2->3->5
Path cost: 9
Visited states: 67
Average time: 0.0037769476572672525
---------------A* alpha = 2-------------------
1->2->3->4->4->4->3->2->3->5
Path cost: 9
Visited states: 42
Average time: 0.0015048980712890625
---------------A* alpha = 4-------------------
1->2->3->4->4->4->3->2->3->5
Path cost: 9
Visited states: 36
Average time: 0.0016667842864990234
---------------------------------------------------------------------------
---------------------------------------------------------------------------
 **  Test Number 1  ** 
---------------BFS--------------

# Comparing

input_0.txt:

| Algorithm | Cost | Visited States | Run Time |                       Path                          |                            
| :-: | :--: | :------------: | :------: |  :------------------------------------------------: |                          
| BFS | 9    | 89             | 0.0020   |              1->2->3->4->4->4->3->2->3->5           |  
| IDS | 9    | 2123           | 0.0186   |              1->2->3->4->4->4->3->2->3->5           |  
| A*  | 9    | 67             | 0.0011   |              1->2->3->4->4->4->3->2->3->5           |  
| A* Alpha=1.2 | 9 | 42       | 0.0006   |              1->2->3->4->4->4->3->2->3->5           |  
| A* Alpha=5   | 9 | 36       | 0.0011   |              1->2->3->4->4->4->3->2->3->5           |  

input_1.txt:

| Algorithm | Cost | Visited States | Run Time |                       Path                        |                            
| :-: | :--: | :------------: | :------: |  :------------------------------------------------: |                          
| BFS | 10    | 7031             | 1.9673   |              6->4->10->5->3->8->9->1->7->9->2           |  
| IDS | -    | -           | -   |              -          |  
| A*  | 10    | 660             | 0.0856   |              6->4->10->9->1->3->8->9->7->9->2           |  
| A* Alpha=1.2 | 10 | 220| 0.0194   |              6->4->10->9->1->3->8->9->7->9->2           |  
| A* Alpha=5   | 10 | 89       | 0.0047   |              6->4->10->9->1->3->8->9->7->9->2           |  

# Conclusion
BFS and IDS will always give us the optimal solution.  
A* with a consistent heuristic will also give the optimal solution.  
But weighted A* may not always give us the optimal solution, but if it does, it is faster than the rest of the algorithms.

As a general speed comparison: Weighted A* > A* > BFS > IDS  